In [ ]:
import numpy as np
import pandas as pd
from itertools import product

## display setup
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 100

In [ ]:
df = pd.read_csv('Police_Incidents_20240429.csv')
print(df.shape)
df.head(1)

(86, 3)


/var/folders/0d/wr3k3hx10rz8pn02h5xblk3m0000gn/T/ipykernel_71810/3788762881.py:4: DtypeWarning: Columns (3,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Police_Incidents_20240429.csv')


(1215789, 86)


,Incident Number w/year,Year of Incident,Service Number ID,Watch,Call (911) Problem,Type of Incident,Type Location,Type of Property,Incident Address,Apartment Number,Reporting Area,Beat,Division,Sector,Council District,Target Area Action Grids,Community,Date1 of Occurrence,Year1 of Occurrence,Month1 of Occurence,Day1 of the Week,Time1 of Occurrence,Day1 of the Year,Date2 of Occurrence,Year2 of Occurrence,Month2 of Occurence,Day2 of the Week,Time2 of Occurrence,Day2 of the Year,Date of Report,Date incident created,Offense Entered Year,Offense Entered Month,Offense Entered Day of the Week,Offense Entered Time,Offense Entered Date/Time,CFS Number,Call Received Date Time,Call Date Time,Call Cleared Date Time,Call Dispatch Date Time,Special Report (Pre-RMS),Person Involvement Type,Victim Type,Victim Race,Victim Ethnicity,Victim Gender,Responding Officer #1 Badge No,Responding Officer #1 Name,Responding Officer #2 Badge No,Responding Officer #2 Name,Reporting Officer Badge No,Assisting Officer Badge No,Reviewing Officer Badge No,Element Number Assigned,Investigating Unit 1,Investigating Unit 2,Offense Status,UCR Disposition,Modus Operandi (MO),Family Offense,Hate Crime,Hate Crime Description,Weapon Used,Gang Related Offense,Drug Related Istevencident,RMS Code,Criminal Justice Information Service Code,Penal Code,UCR Offense Name,UCR Offense Description,UCR Code,Offense Type,NIBRS Crime,NIBRS Crime Category,NIBRS Crime Against,NIBRS Code,NIBRS Group,NIBRS Type,Update Date,X Coordinate,Y Cordinate,Zip Code,City,State,Location1
0,119031-2022,2022,119031-2022-01,2,09V - UUMV,UNAUTHORIZED USE OF MOTOR VEH - AUTOMOBILE,Outdoor Area Public/Private,Outdoor Area Public/Private,4839 HAAS DR,NaN,4309.0,732.0,SOUTH CENTRAL,730.0,D4,Ledbetter Sunnyvale,NaN,2022-07-02 00:00:00.0000000,2022,July,Sat,14:22,183,2022-07-02 00:00:00.0000000,2022.0,July,Sat,14:50,183.0,2022-07-02 14:52:00.0000000,2022-07-02 16:28:37.0000000,2022,July,Sat,16:28,183,22-1270590,2022-07-02 14:52:17.0000000,2022-07-02 14:52:17.0000000,2022-07-02 17:21:03.0000000,2022-07-02 16:03:50.0000000,NaN,Victim,Individual,Hispanic or Latino,Hispanic or Latino,Female,12040,"RODRIGUEZ,DAVID",NaN,NaN,12040,125759,129123,C757,Investigations,Special Investigations / Auto Theft,Suspended,Suspended,UNK SUSPECT TOOK COMP'S VEH AND FLED LOC.,False,NaN,NaN,NaN,NaN,No,FS-24110003-G13,24110003.0,PC 31.07,NaN,NaN,NaN,NaN,UUMV,MOTOR VEHICLE THEFT,PROPERTY,240,A,Not Coded,2022-07-23 20:37:40.0000000,2.501861e+06,6.942006e+06,75216.0,DALLAS,TX,"4839 HAAS DR\nDALLAS, TX 75216\n(32.700112, -9..."


In [ ]:
## Truncate to from 2023
df_23_24 = df[(df['Year1 of Occurrence'] >= 2023) & (df['Year1 of Occurrence'] <= 2024) & (~df['Location1'].isnull())]

df_23_24 = df_23_24[['Incident Number w/year', 'Division', 
                     'Date1 of Occurrence', 'Day1 of the Week', 'Time1 of Occurrence',
                     'X Coordinate', 'Y Cordinate', 'Zip Code', 'Location1']]

## Filter out edge cases
df_23_24 = df_23_24[(~df_23_24.longitude.isnull()) & (~df_23_24.latitude.isnull())]
df_23_24 = df_23_24[(df_23_24.longitude >= -98.0) & (df_23_24.longitude <= -95.0) & (df_23_24.latitude >= 31.0) & (df_23_24.latitude <= 35.0)]

df_23_24.shape

# plt.lon_bin((-98.0, -95.0))
# plt.lat_bin((31.0, 35.0))

df_23_24.head(1)

(188780, 86)

In [ ]:
def clean_up_loc(loc_string):
    """
    Process and parse the longitude and latitude from orinal dataset
    """
    try:
        res = loc_string.split('\n')[-1][1:-1].split(',')
        return float(res[1]), float(res[0])
    except:
        return None, None
    
# geometry = [clean_up_loc(row['Location1']) for _, row in df_23_24.iterrows()]

df_23_24[['longitude', 'latitude']] = df_23_24['Location1'].apply(lambda x: pd.Series(clean_up_loc(x)))

# geo_df = gpd.GeoDataFrame(df_23_24, #specify our data
#                           crs={'init':'epsg:4326'}, #specify our coordinate reference system
#                           geometry=geometry) #specify the geometry list we created
df_23_24.head(1)

,Incident Number w/year,Division,Date1 of Occurrence,Day1 of the Week,Time1 of Occurrence,X Coordinate,Y Cordinate,Zip Code,Location1,longitude,latitude
31,803086-2023,CENTRAL,2023-03-16 00:00:00.0000000,Thu,13:45,2489804.475,6.971231e+06,75202.0,"1015 ELM ST\nDALLAS 75202\n(32.7805, -96.80355)",-96.80355,32.7805


In [ ]:
def count_events_by_grid_and_time(data, lon_step=0.01, lat_step=0.01, start_date = None, end_date = None):
    """
    Create grids and time pairs.
    """
    # Create grid bins for longitude and latitude, pick one on flooring/ceiliing
    
    data['lon_bin'] = (data['longitude'] // lon_step) * lon_step
    data['lon_bin'] = data['lon_bin'].round(2)
    data['lat_bin'] = (data['latitude'] // lat_step) * lat_step
    data['lat_bin'] = data['lat_bin'].round(2)
    data['date'] = pd.to_datetime(data['Date1 of Occurrence']).dt.date
    
    if start_date is None:
        start_date = data['date'].min()
    if end_date is None:
        end_date = data['date'].max()
    full_dates = pd.date_range(start=start_date, end=end_date).date
    
    # Get the unique values for lon_bin, lat_bin, and time_bin
    lon_bins = np.arange(data['lon_bin'].min(), data['lon_bin'].max() + lon_step, lon_step)
    lat_bins = np.arange(data['lat_bin'].min(), data['lat_bin'].max() + lat_step, lat_step)

    # Create a full Cartesian product of all possible bins
    full_grid = pd.DataFrame(
        product(lon_bins, lat_bins, full_dates),
        columns=['lon_bin', 'lat_bin', 'date']
    )
    full_grid['lon_bin'] = full_grid['lon_bin'].round(2)
    full_grid['lat_bin'] = full_grid['lat_bin'].round(2)
    print('prepare done, full_grid size:', full_grid.shape[0])

    # Group by grid (lon_bin, lat_bin, time_bin) and count unique event_id values
    grouped = data.groupby(['lon_bin', 'lat_bin', 'date'])['Incident Number w/year'].nunique().reset_index(name='unique_event_count')

    # Merge the full grid with the grouped data to fill in missing bins with zero counts
    result = full_grid.merge(grouped, on=['lon_bin', 'lat_bin', 'date'], how='left').fillna(0)

    # Ensure unique_event_count is integer type
    result['unique_event_count'] = result['unique_event_count'].astype(int)

    return result, grouped

In [141]:
# prepare done, full_grid size: 25852230
final_result, grouped_df = count_events_by_grid_and_time(df_23_24, lon_step=0.05, lat_step=0.05, start_date = None, end_date = None)
print(final_result.shape, grouped_df.shape)

prepare done, full_grid size: 248320
(248320, 4) (20950, 4)


In [142]:
print(final_result.shape)
final_result.head(1)

(248320, 4)


,lon_bin,lat_bin,date,unique_event_count
0,-97.8,32.45,2023-01-01,0


In [143]:
final_result.to_csv('processed_incident_count_005.csv')

In [ ]:
final_result_001, grouped_df_001 = count_events_by_grid_and_time(df_23_24, lon_step=0.01, lat_step=0.01, start_date = None, end_date = None)
print(final_result.shape, grouped_df.shape)

final_result_001.to_csv('processed_incident_count_001.csv')